# Building a RAG application from scratch using `LangChain` and `Bedrock`

Before we get started, I would like to thank to [Santiago](https://www.youtube.com/@underfitted) for his wonderful tutorial on `LangChain`, this notebook is based on his tutorial. 
Here is a high-level overview of the system we want to build:

<img src='./images/IMG_0354.jpg' width="1200">

## Setup

Create an `.env `file in the project directory using env.example as a reference. Populate the .env file with your Aurora PostgreSQL DB cluster details (this we will need later on):

```
PGVECTOR_DRIVER='psycopg2'
PGVECTOR_USER='<<Username>>'
PGVECTOR_PASSWORD='<<Password>>'
PGVECTOR_HOST='<<Aurora DB cluster host>>'
PGVECTOR_PORT=5432
PGVECTOR_DATABASE='<<DBName>>'
```

## Loading the `env` variables

Let's start by loading the environment variables we need to use.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Setting up the model
Let's define the LLM model that we'll use as part of the workflow.

In [40]:
import langchain
langchain.__version__

'0.2.1'

In [2]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage


model = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs={"temperature": 0.1})

We can test the model by asking a simple question.

In [41]:
model.invoke("Who won the ICC Criket World Cup 2019?")

AIMessage(content="The 2019 ICC Cricket World Cup was won by England. It was hosted in England and Wales.\n\nIn the final at Lord's Cricket Ground in London, England defeated New Zealand in a dramatic match that went to a Super Over tie-breaker after the scores were tied at the end of the regulation 50 overs per side.\n\nEngland scored 241/8 in their 50 overs, which New Zealand also scored to tie the match. In the Super Over, both teams scored 15 runs each. However, England was awarded the World Cup on a controversial boundary countback rule, having scored more boundaries (fours and sixes) during the match.\n\nIt was England's first ever Cricket World Cup title victory. New Zealand were the runners-up for the second consecutive World Cup after 2015. The player of the tournament was Kane Williamson of New Zealand.", additional_kwargs={'usage': {'prompt_tokens': 20, 'completion_tokens': 190, 'total_tokens': 210}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-

In [3]:
messages = [
    HumanMessage(
        content="Who won the ICC Criket World Cup 2019?"
    )
]

model.invoke(messages)

AIMessage(content="The 2019 ICC Cricket World Cup was won by England. It was hosted in England and Wales.\n\nIn the final at Lord's Cricket Ground in London, England defeated New Zealand in a dramatic match that went to a Super Over tie-breaker after the scores were tied at the end of the regulation 50 overs per side.\n\nEngland scored 241/8 in their 50 overs, which New Zealand also scored to tie the match. In the Super Over, both teams scored 15 runs each. However, England was awarded the World Cup on a controversial boundary countback rule, having scored more boundaries (fours and sixes) during their innings.\n\nIt was England's first ever Cricket World Cup title. New Zealand were the runners-up for the second consecutive World Cup after 2015. The player of the tournament was Kane Williamson of New Zealand.", additional_kwargs={'usage': {'prompt_tokens': 20, 'completion_tokens': 189, 'total_tokens': 209}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0

The result from the model is an `AIMessage` instance containing the answer. We can extract this answer by chaining the model with an [output parser](https://python.langchain.com/docs/modules/model_io/output_parsers/).

Here is what chaining the model with an output parser looks like:

<img src='./images/IMG_0355.jpg' width="1200">

For this example, we'll use a simple `StrOutputParser` to extract the answer as a string.

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser

# model.invoke("our question")
chain.invoke("Who won the ICC Criket World Cup 2019?")

"The 2019 ICC Cricket World Cup was won by England. It was hosted in England and Wales.\n\nIn the final at Lord's Cricket Ground in London, England defeated New Zealand in a dramatic match that went to a Super Over tie-breaker after the scores were tied at the end of the regulation 50 overs per side.\n\nEngland scored 241/8 in their 50 overs, which New Zealand also scored to tie the match. The Super Over also ended in a tie with both teams scoring 15 runs each.\n\nHowever, England was awarded the World Cup for the first time in their history based on the tie-breaker rule of scoring more boundaries (fours and sixes) in the match. It was an incredibly close and thrilling final.\n\nThis was England's first ever Cricket World Cup title after making the finals in 1979, 1987 and 1992 previously. New Zealand were the runners-up for the second consecutive World Cup after 2015."

## Introducing prompt templates

We want to provide the model with some context and the question. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) are a simple way to define and reuse prompts.

In [5]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
response = prompt.format(context="Rashmi is Reyaan's mom", question="Who is Rashmi's son ?")

print(response)

Human: 
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: Rashmi is Reyaan's mom

Question: Who is Rashmi's son ?



We can now chain the prompt with the model and the output parser.

<img src='./images/IMG_0356.jpg' width="1200">

In [6]:
chain = prompt | model | parser

response = chain.invoke({
                            "context": "Rashmi is Reyaan's mom",
                            "question": "Who is Rashmi's son ?"
                        })

print(response)

Based on the given context that Rashmi is Reyaan's mom, the answer to "Who is Rashmi's son?" is Reyaan.


## Combining chains

We can combine different chains to create more complex workflows. For example, let's create a second chain that translates the answer from the first chain into a different language.

Let's start by creating a new prompt template for the translation chain:

In [7]:
translation_prompt = ChatPromptTemplate.from_template(
                                                        "Translate {answer} to {language}"
                                                    )

We can now create a new translation chain that combines the result from the first chain with the translation prompt.

Here is what the new workflow looks like:

<img src='./images/IMG_0357.jpg' width="1200">

In [8]:
from operator import itemgetter

translation_chain = (
                        {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
                    )

translation_chain.invoke(
        {
            "context": "John's brother is named Michael. He also has two sisters, Alice and Viktoria.",
            "question": "How many siblings does John have in total?",
            "language": "Hindi"
        }
)

'दिए गए संदर्भ के आधार पर, जॉन के कुल तीन भाई-बहन हैं: एक भाई माइकल और दो बहनें एलिस और विक्टोरिया।'

## Transcribing the YouTube Video

The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using Amazon Transcribe

In [9]:
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=lB_0hR5s41Y&ab_channel=BeerBiceps"
S3_BUCKET = 'ml-dl-demo-data'

In [10]:
from utils import transcribe_video

transcribe_video(s3_bucket_name=S3_BUCKET, youtube_video_url=YOUTUBE_VIDEO)

Transcription file already exists.


Let's read the transcription and display the first few characters to ensure everything works as expected.

In [11]:
import json

with open("transcription.txt", "r") as file:
    transcription = json.loads(file.read())
    transcription = transcription['results']['transcripts'][0]['transcript']

transcription[0:100]

"You're a multibillionaire European founder who's moved to Gandhinagar. Yes. Why did you choose Gujar"

## Using the entire transcription as context

If we try to invoke the chain using the transcription as context, the model will return an error because the context is too long.

Large Language Models support limitted context sizes. The video we are using is too long for the model to handle, so we need to find a different solution.

In [12]:
len(transcription)

114926

In [13]:
chain.invoke({"context": transcription,
              "question": "What matters when selecting a location for a business in India ?"
            })

'According to Fabian, the most important factor when selecting a location for a business in India is not the location itself, but finding the right person/director to lead the operations there. He says:\n\n"The key to succeed a new company is not the location, the key is the director. This is the most important thing. You can have a good director in a bad location. It will work. You can have a good location, average director, it\'s gonna be painful."\n\nHe explains that when expanding to a new location, they first identify the best person/employee within their company who can take the lead, and then open the office wherever that person is based or prefers to be. The location itself is secondary to having the right leadership in place.\n\nHe gives examples of opening offices in places like Buffalo, New York and Gandhinagar, Gujarat not because those were specifically targeted locations, but because they had the right people to lead those offices there. Finding and empowering the right l

## Splitting the transcription



Since we can't use the entire transcription as the context for the model, a potential solution is to split the transcription into smaller chunks. We can then invoke the model using only the relevant chunks to answer a particular question:

<img src='./images/IMG_0358.jpg' width="1200">

Let's start by loading the transcription in memory:

In [14]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_documents = loader.load()

There are many different ways to split a document. For this example, we'll use a simple splitter that splits the document into chunks of a fixed size. Check [Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/) for more information about different approaches to splitting documents.

For illustration purposes, let's split the transcription into chunks of 100 characters with an overlap of 20 characters and display the first few chunks:

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(page_content='{"jobName":"Multi-BillionairesJourneyInIndia-LeadershipCultureAndOpportunityOdooTRS386.mp41717446851', metadata={'source': 'transcription.txt'}),
 Document(page_content='TRS386.mp41717446851","accountId":"507922848584","status":"COMPLETED","results":{"transcripts":[{"tr', metadata={'source': 'transcription.txt'}),
 Document(page_content='{"transcripts":[{"transcript":"You\'re', metadata={'source': 'transcription.txt'}),
 Document(page_content="a multibillionaire European founder who's moved to Gandhinagar. Yes. Why did you choose Gujarat? In", metadata={'source': 'transcription.txt'}),
 Document(page_content='choose Gujarat? In India? We have a ruler to do is we never go to tier one cities. We always go to', metadata={'source': 'transcription.txt'})]

For our specific application, let's use 1000 characters instead:

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## Finding the relevant chunks



Given a particular question, we need to find the relevant chunks from the transcription to send to the model. Here is where the idea of **embeddings** comes into play.

An embedding is a mathematical representation of the semantic meaning of a word, sentence, or document. It's a projection of a concept in a high-dimensional space. Embeddings have a simple characteristic: The projection of related concepts will be close to each other, while concepts with different meanings will lie far away. 

To provide with the most relevant chunks, we can use the embeddings of the question and the chunks of the transcription to compute the similarity between them. We can then select the chunks with the highest similarity to the question and use them as the context for the model:

<img src='./images/IMG_0359.jpg' width="1200">

Let's generate embeddings for an arbitrary query:

In [17]:
from langchain_community.embeddings import BedrockEmbeddings

embeddings = BedrockEmbeddings()
embedded_query = embeddings.embed_query("Berlin is in Germany")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[1.2890625, 0.4453125, 0.28320312, 0.3984375, 0.050048828, -0.123046875, 0.58984375, -0.0007247925, -0.23535156, 0.48046875]


To illustrate how embeddings work, let's first generate the embeddings for two different sentences:

In [18]:
sentence1 = embeddings.embed_query("Welcome to Frankfurt")
sentence2 = embeddings.embed_query("This is a table")

We can now compute the similarity between the query and each of the two sentences. The closer the embeddings are, the more similar the sentences will be.

We can use [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) to calculate the similarity between the query and each of the sentences:

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.6138958023127881, 0.2699050016319834)

## Setting up a Vector Store

We need an efficient way to store document chunks, their embeddings, and perform similarity searches at scale. To do this, we'll use a **vector store**.

A vector store is a database of embeddings that specializes in fast similarity searches. 

<img src='./images/IMG_0360.jpg' width="1200">

To understand how a vector store works, let's create one in memory and add a few embeddings to it:

### Storing vectors in Amazon Aurora using `pgvector`

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.1em;">
<b>Prerequisite:</b>
<ol>
    <li>Have an <b>Aurora cluster ready</b>.</li>
    <li>Create the <b>pgvector extension</b> on your Aurora PostgreSQL database (DB) cluster:
        <pre style="font-size: 1.1em;"><code>
        CREATE EXTENSION vector;
        </code></pre>
    </li>
</ol>
</div>


We can connect to the Aurora cluster and check 


```sql
-- SHOW the current database
SELECT current_database();

-- SHOW all the tables in the database
SELECT table_name
FROM postgres.information_schema.tables
WHERE table_schema = 'public';
```

In [20]:
from langchain_community.vectorstores.pgvector import PGVector, DistanceStrategy

# Loading all env variables 
load_dotenv()

COLLECTION_NAME = 'rag-intro-on-aws'

# Connection String
CONNECTION_STRING = PGVector.connection_string_from_db_params(driver = os.getenv("PGVECTOR_DRIVER"),
                                                              user = os.getenv("PGVECTOR_USER"),                                      
                                                              password = os.getenv("PGVECTOR_PASSWORD"),                                  
                                                              host = os.getenv("PGVECTOR_HOST"),                                            
                                                              port = os.getenv("PGVECTOR_PORT"),                                          
                                                              database = os.getenv("PGVECTOR_DATABASE"),
                                                              )  

# Text Embedding model
embeddings = BedrockEmbeddings()

# Creating the VectorDB store instance   
vectorstore1 = PGVector(   collection_name=COLLECTION_NAME,
                           connection_string=CONNECTION_STRING,
                           embedding_function=embeddings,
                           distance_strategy = DistanceStrategy.EUCLIDEAN,
                           use_jsonb = True
                          )

/opt/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See https://github.com/langchain-ai/langchain-postgres for details aboutthe new implementation.
  warn_deprecated(


In [21]:
vectorstore1.add_texts([
                    "Color of the bird is red"
                    "The cat slept by the fire.",
                    "We went to the park after school.",
                    "I finished my homework early.",
                    "The bird sang a beautiful song.",
                    "She read a book before bed.",
                    "Mary has two siblings",
                    "Song was in Spanish", 
                    ])

['a9cd8682-46ab-4b6f-a472-545dc5f83692',
 '97f9e8ca-73f3-451e-8478-42a313dbfc59',
 '5a55f20d-49ea-4db7-a2d7-2fbd19bea05a',
 '2847fbf8-8393-4e18-8798-110affceaad1',
 '6202efd7-055c-4849-97c6-7479dc5ec0e5',
 '818f051b-f02b-4089-8b27-a1980ac7569b',
 'a70d4334-52b3-4e47-a97e-203e4391cc7f']

We can now query the vector store to find the most similar embeddings to a given query:

In [22]:
vectorstore1.similarity_search_with_score(query="What the bird was singing", k=3)

[(Document(page_content='The bird sang a beautiful song.'),
  12.719913663751074),
 (Document(page_content='The bird sang a beautiful song.'),
  12.719913663751074),
 (Document(page_content='The bird sang a beautiful song.'),
  12.719913663751074)]

## Connecting the vector store to the chain

We can use the vector store to find the most relevant chunks from the transcription to send to the model. Here is how we can connect the vector store to the chain:

<img src='./images/IMG_0361.jpg' width="1200">

We need to configure a [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/). The retriever will run a similarity search in the vector store and return the most similar documents back to the next step in the chain.

We can get a retriever directly from the vector store we created before: 

In [23]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Whats the color of the bird who was singing?")

[Document(page_content='Color of the bird is redThe cat slept by the fire.'),
 Document(page_content='Color of the bird is redThe cat slept by the fire.'),
 Document(page_content='Color of the bird is redThe cat slept by the fire.'),
 Document(page_content='Color of the bird is redThe cat slept by the fire.')]

Our prompt expects two parameters, "context" and "question." We can use the retriever to find the chunks we'll use as the context to answer the question.

We can create a map with the two inputs by using the [`RunnableParallel`](https://python.langchain.com/docs/expression_language/how_to/map) and [`RunnablePassthrough`](https://python.langchain.com/docs/expression_language/how_to/passthrough) classes. This will allow us to pass the context and question to the prompt as a map with the keys "context" and "question."

In [24]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("Whats the color of the bird who was singing?")

{'context': [Document(page_content='Color of the bird is redThe cat slept by the fire.'),
  Document(page_content='Color of the bird is redThe cat slept by the fire.'),
  Document(page_content='Color of the bird is redThe cat slept by the fire.'),
  Document(page_content='Color of the bird is redThe cat slept by the fire.')],
 'question': 'Whats the color of the bird who was singing?'}

Let's now add the setup map to the chain and run it:



In [25]:
chain = setup | prompt | model | parser
chain.invoke("Whats the color of the bird who was singing?")

'The context provided does not mention a singing bird. All the documents state "Color of the bird is red" and "The cat slept by the fire." There is no information about a singing bird or its color.'

Let's invoke the chain using another example:

In [26]:
chain.invoke("Does Mary has any brother or sister ?")

'Based on the given context that "Mary has two siblings", I can infer that Mary has at least one brother or one sister, since siblings refer to brothers and/or sisters.'

In [27]:
## Loading transcription into the vector store


We initialized the vector store with a few random strings. Let's create a new vector store using the chunks from the video transcription.

## Setting up Aurora

In [28]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores.pgvector import PGVector
import os
from dotenv import load_dotenv

# Loading all env variables
load_dotenv()

# Load the text from the file
loader = TextLoader("transcription.txt")
documents = loader.load()

# Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Initialize the embeddings
embeddings = BedrockEmbeddings()

# Set the collection name
COLLECTION_NAME = "rag-intro-yt"

# Connection String
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.getenv("PGVECTOR_DRIVER"),
    user=os.getenv("PGVECTOR_USER"),
    password=os.getenv("PGVECTOR_PASSWORD"),
    host=os.getenv("PGVECTOR_HOST"),
    port=os.getenv("PGVECTOR_PORT"),
    database=os.getenv("PGVECTOR_DATABASE"),
)

# Create the PGVector instance from the documents
db = PGVector.from_documents(
                                embedding=embeddings,
                                documents=docs,
                                collection_name=COLLECTION_NAME,
                                connection_string=CONNECTION_STRING,
                                use_jsonb = True
                            )

Let's now run a similarity search on Aurora to make sure everything works:

In [29]:
db.similarity_search("Can you share the detail of the speaker's journey from starting as a coder to becoming a successful entrepreneur, including the pivot in his business model?")[:3]

[Document(page_content="that. You like how you're working, right? The reason is that um uh public companies tend to refocus on the short term, you know, you have to uh publish earning codes with the, the sales number and if the numbers are good, everyone is happy, they buy the shares. If numbers are bad, people are not happy and your employees are frustrated because they have shares and it's bad. So public companies have a tendency to focus on the short term to saves, saves of the moments of the quarter and so on. I don't want that the success of FU is always to build for the long term. And I don't want to see on to look for the short term or the sales number of the quarter. One, the other thing is I'm so much focused on productivity and efficiency. Uh When you get public, you need extra layers of reporting transparency, uh anything like that. Uh So second reason I don't want that I want to be super efficient, decide right away instead of asking the board of directors. Um And third, I 

In [44]:
len([0.19355178,-0.08362444,0.049161315,-0.004458597,0.4079286,-0.08792504,0.10263547,0.0006831645,-0.2195752,0.16911036,0.050681725,0.050152235,-0.048521034,-0.09615231,0.08053398,-0.04728475,0.02478852,0.16927248,-0.03453377,0.12205102,-0.22682455,-0.08031672,-0.4397703,-0.34945098,-0.05380631,-0.26713648,0.16850244,-0.45341694,-0.024653953,0.085642256,-0.047029253,0.26540935,0.08999313,-0.051907357,0.041951753,-0.17402966,0.05673165,-0.27109727,-0.19441305,-0.13837372,0.08247412,0.295728,0.15918922,0.28445876,0.07128833,0.13714461,0.030616384,-0.21032792,-0.02544101,0.33628786,-0.16137393,0.06770626,0.37295318,0.32126996,0.20138295,-0.14569482,0.06364783,-0.084431395,-0.3449368,-0.079366535,0.0016877316,0.23757048,0.3083524,0.33201173,0.20518261,-0.2866155,-0.45871317,-0.37895846,0.16553128,-0.1494095,0.16908836,-0.007247297,0.07378427,0.31937367,0.04495919,-0.12163822,-0.093258604,-0.18798971,0.16448838,0.21381141,0.17376457,0.15485172,-0.18482284,0.2776282,-0.017378978,-0.033287097,0.15411898,-0.16158065,0.00013931945,-0.005512705,0.15415132,0.26156157,-0.032070644,0.038298663,0.06374921,0.29249933,-0.14012428,-0.21980989,0.40956163,-0.32664955,-0.05693525,0.0019605886,-0.017785255,0.1873138,-0.01155537,-0.02244435,0.12207446,-0.106745325,-0.004468124,0.4013286,0.16791026,0.062429916,0.2054484,-0.067060605,-0.22272757,0.028570984,-0.3502803,0.32123554,0.015531906,-0.67756593,-0.030637108,0.036299497,-0.24510682,0.055966776,0.1788907,-0.07713745,-0.19199757,0.025081482,-0.17815635,-0.07457786,-0.08944234,0.10590167,0.033636875,0.28832853,0.4488837,-0.31013218,-0.24526852,-0.10484383,-0.046187256,-0.046318915,-0.07231455,-0.028890284,-0.032358013,0.2672493,0.052554816,0.23204866,0.06919329,-0.35887158,0.17202015,-0.13723668,-0.11075753,0.27195668,-0.117374934,0.0831119,-0.070854895,0.032761496,0.043760158,-0.012161624,-0.19433841,0.2177465,-0.111157686,-0.007472882,0.13090292,-0.15421571,0.0916055,0.11667578,-0.020831726,0.11309879,0.117864355,-0.4082819,0.058176585,0.06665152,0.0029617208,-0.23794162,0.07924022,-0.13909006,-0.19494177,0.18180235,0.075063854,-0.054935478,-0.024471907,-0.051460076,-0.02342423,0.33661923,-0.072135,0.011239801,0.14604191,-0.2456488,-0.303759,0.24490777,0.08027011,0.11047907,0.036199573,0.035874054,-0.42679554,0.05512839,0.010638518,-0.13245836,0.15500721,-0.38443717,0.0002468354,0.18391594,-0.021262938,0.18865155,-0.26142722,0.07111832,-0.20080696,-0.15448597,0.17373088,0.14247589,-0.20645401,0.13643678,-0.15091684,-0.2953405,0.18984665,0.012499584,0.2002123,-0.053764846,-0.023569364,-0.25330743,0.15946083,-0.14061753,-0.23216146,-0.3955165,-0.19351447,0.11355576,-0.48885185,0.03494084,-0.07111116,0.117795,0.007916998,-0.09210497,-0.012144832,-0.18646953,0.08045548,0.21677323,-0.5729382,-0.10826697,-0.044855684,0.46128833,-0.07872068,-0.36763698,-0.02240206,-0.19261844,0.22427472,0.15221064,0.04868029,-0.02914873,-0.05705913,0.067288265,0.09073585,-0.010486954,-0.52411425,-0.11246154,-0.40252206,-0.15308228,-0.25814342,0.17564192,0.19336599,0.034555588,-0.06267001,0.12478271,-0.0048521273,0.012175516,0.07554239,-0.48920846,0.05220711,0.21302497,0.031598743,-0.39734894,0.00012982402,-0.11375191,0.28567126,0.25330496,0.12005659,0.5485851,-0.033756718,0.24272284,0.1654467,-0.11177576,0.55417496,-0.0021160145,0.17382064,-0.1544696,0.20297903,-0.57029796,0.18555102,0.5494128,-0.28566277,-0.14807591,-0.13542174,0.16831478,0.24604729,0.031842988,0.35818356,-0.17025277,0.24215308,-0.009459594,-0.14041723,-0.06633057,0.16387464,-0.052826017,0.2714421,0.33476603,-0.22163987,-0.18519376,0.091173925,0.05053493,-0.007543789,0.2625734,-0.094086714,0.2589371,-0.15614052,0.1962646,-0.07959487,0.044794984,0.08784646,-0.1741522,-0.08403807,-0.12911919,0.036458958,-0.19098826,-0.14064585,-0.1500929,0.1151589,0.17611112,-0.23384753,-0.034208298,0.32291046,-0.21695611,-0.19479497,-0.2659443,-0.1414668,-0.054365713,0.078789726,-0.3613107,0.23614109,0.11740728,0.10477623,-0.17310494,0.058060475,-0.063552245,0.18890907,-0.3721495,-0.07338732,0.17225598,0.059656978,0.116717674,0.15800573,0.079366334,0.016810354,-0.016369265,0.30977887,-0.19772425,-0.12823032,0.1983425,0.13914627,0.040231057,-0.31363788,-0.19998918,-0.10082015,-0.121112406,0.14084746,-0.3381195,0.07975697,0.047642563,-0.13077891,-0.67033744,0.044763677,-0.07801553,0.072637126,0.25382993,-0.1904616,-0.29494095,-0.097912095,0.6852267,-0.04899731,-0.04154454,0.070548445,0.3599713,-0.310381,0.28519025,-0.0009452556,0.034242302,-0.22797033,0.07469915,-0.10746126,-0.38205197,0.029823307,-0.82779324,0.27077195,-0.18379445,0.160867,-0.37351215,-0.28570363,0.08081292,-0.123316415,0.08894763,-0.09140377,0.05553095,-0.14441305,0.33530843,0.14099042,0.17095275,0.03174761,-0.1238414,0.10277512,-0.123075895,0.19733487,-0.045525648,-0.016301543,-0.1059535,0.036969833,-0.17383051,0.019313129,-0.11639256,-0.016375355,-0.04397973,-0.0893988,0.31471437,0.11451657,-0.11699171,-0.10360775,0.08459747,0.11570338,0.106387615,-0.02643789,-0.15057266,-0.003192179,-0.20831218,-0.1638025,-0.048616644,-0.041170083,0.07387664,-0.18631485,0.09456664,-0.20039539,0.02178999,-0.16322733,-0.07803708,0.19657104,-0.12560916,0.11512396,-0.37981313,0.04911384,0.025246212,-0.19350117,0.1537354,0.1288656,0.20239682,-0.05267964,0.057841323,0.06594907,-0.40608576,-0.11904857,-0.64553976,-0.13828623,-0.063143164,0.36915264,0.01405598,0.0038151247,0.13573015,0.21153048,-0.3404865,0.03395242,0.016978089,0.46331275,-0.00701176,0.0010402142,0.12301577,-0.31627107,-0.023603577,-0.18435177,0.08211999,-0.30009037,-0.23892607,-0.030108083,0.037438076,0.026589762,0.0894921,0.059717678,0.1524383,0.060112294,-0.18706064,-0.12889962,0.23557132,0.095532276,-0.045130044,-0.12618515,0.059197515,0.056447797,0.35793275,-0.14981297,-0.25970924,-0.20514406,0.061926816,-0.00050022907,-0.041905105,0.25465846,-0.24197268,0.09091361,0.047915004,0.048704132,-0.043504924,0.076253146,0.026611641,0.19387108,-0.23941995,0.044988222,-0.31102332,-0.15617493,0.2970415,0.03496057,-0.0891954,0.029828366,0.11628724,-0.056123726,0.5105046,-0.030607667,-0.003076069,-0.03505838,0.49123746,-0.2724286,-0.2510682,-0.0454933,-0.19251344,0.099675305,0.16148901,0.012893526,0.47026935,-0.12565601,-0.3540323,-0.3257514,0.21508841,0.47222498,0.5111863,-0.29438445,-0.14802551,0.1373051,0.007449452,-0.09515741,0.3140376,-0.35559976,-0.033757545,-0.2512465,-0.1499577,-0.063297,-0.10316853,0.07697759,0.47310743,0.30575487,-0.019839976,-0.5931727,-0.05643785,0.48675936,0.16257796,-0.10702462,-0.16270402,0.10863387,0.081830956,-0.059191294,-0.13400054,-0.11639858,-0.24813603,0.19765085,0.15427738,0.09753091,0.022481255,0.14062251,0.14913778,0.08511354,0.013859726,-0.18246102,0.15241963,0.19860294,0.040288284,0.4765758,-0.09610328,0.16351596,-0.091562584,-0.1257914,0.3894987,0.38745564,0.31646222,-0.17403962,-0.007892324,-0.26164162,-0.20080736,-0.14165588,-0.14021653,0.10888783,-0.2616101,-0.18499245,-0.10398929,0.028521845,-0.14593866,0.24380472,-0.36394638,-0.18487759,-0.27835804,0.03703162,0.41708156,0.008209085,-0.12861845,-0.120162316,-0.13574882,-0.31429014,-0.05337453,-0.009672893,-0.04326441,0.08672787,0.21518916,0.13200802,0.15753424,-0.390309,0.09520945,0.3189482,-0.0068960646,0.2905155,0.011749953,0.24285677,-0.20390293,0.05791948,0.13102701,-0.06011571,0.09097841,-0.14726396,-0.42657244,0.1292778,0.42697632,-0.10060812,-0.010035163,-0.073505506,-0.078706585,-0.12059074,0.19668008,0.056303903,0.14967366,0.26225618,-0.09725515,-0.20352308,0.4545747,-0.0899443,-0.017677233,0.23972017,-0.32668605,0.3948638,0.6149797,-0.29236996,0.0297101,0.037010476,0.5269224,-0.24352731,0.087842725,-0.05518138,-0.38134867,0.2868585,0.05144535,-0.2581434,0.030791834,-0.26561224,-0.36295447,-0.21745661,0.2581915,0.18122117,-0.07307818,-0.02153911,0.18843628,-0.08217309,-0.11804339,0.12503964,-0.4076947,0.15887904,0.17545137,-0.30578616,0.017495189,-0.01432789,-0.021609193,-0.43153033,0.18831857,0.09168823,-0.14593492,-0.11547488,-0.2836037,0.11103328,-0.36326963,-0.081237555,-0.15527624,0.30058843,0.047031745,0.06920386,-0.110971496,-0.2466801,-0.077187,0.18571855,0.015617328,0.018863205,0.046054352,0.09241588,0.020316724,-0.24685343,-0.020821305,-0.09680844,-0.37195542,-0.00016980711,-0.07162214,0.15643412,0.018659808,-0.05438271,-0.3617652,0.016696109,0.16878523,0.12520671,0.08981327,0.067432776,-0.50993395,0.18392465,0.2529276,0.042057082,0.10761595,-0.20548534,0.14255077,0.19049996,0.10240388,-0.043322053,0.016784174,0.26353586,-0.27369583,0.28012952,-0.043568578,0.20908388,0.05455159,0.45363754,0.26871932,0.00538457,-0.0034912603,-0.04974497,-0.045608163,-0.16137704,-0.24668756,0.049836613,0.24460176,-0.09665195,-0.14895366,0.34069133,0.24090864,-0.032804623,0.21099971,0.10918009,0.03876891,0.20893916,-0.32069236,0.16946426,0.39063078,0.045058303,0.047726125,0.04929474,0.072303355,0.20994195,0.1767972,-0.24838525,0.11021409,0.26403096,0.10802564,-0.0037662974,0.10082396,-0.019287005,-0.20121747,0.043351702,-0.0021683425,0.31021345,0.09062873,0.047522724,-0.19491689,-0.008127033,0.05034428,0.1355423,0.05886163,-0.054061025,0.037868854,0.0822925,-0.01622421,-0.15046315,0.1280715,-0.42722017,-0.08066116,0.057268858,0.31319168,-0.022337675,0.41702,-0.1604119,0.088591635,0.20361513,-0.06546923,0.36307222,-0.022331972,-0.117289394,-0.052917242,0.19474936,-0.10610922,-0.2460983,0.080573864,0.52344745,0.19038117,-0.15809862,0.047282416,-0.26666585,0.17062683,0.28176087,0.2077901,-0.25785312,-0.043579776,0.42230624,0.24005938,-0.32731557,0.22998315,0.03369659,-0.39547753,0.12660852,0.40756866,-0.16547818,-0.14392167,0.12381899,-0.43165222,0.1509521,0.001372643,0.28542206,-0.05744427,-0.59653664,0.056689452,-0.10529189,-0.030949151,-0.12939763,-0.07492888,0.07369792,0.0048434176,-0.036528,-0.0010661326,0.23216479,-0.11684526,-0.10945005,-0.20245072,0.34272408,-0.07196456,0.2601156,0.14845407,0.10184469,-0.34132415,-0.096432954,0.113893315,-0.042574387,0.14104256,0.2871873,-0.09351115,-0.11412968,-0.16168185,-0.15149656,-0.11559172,0.1290154,-0.029607052,0.21269323,0.18493563,0.19016314,0.0058755483,-0.109238565,-0.06890856,0.022663403,-0.0051760953,-0.11089197,0.012915712,-0.08156764,-0.08826217,0.10111071,-0.028875975,0.029193206,0.2455337,-0.09252878,0.11763534,-0.06836973,0.16435027,-0.07498417,-0.20559086,0.3201458,-0.038096823,0.237048,0.30674553,0.55826527,0.6140583,0.053870693,0.39595482,-0.642418,0.07120737,0.17086215,-0.12381976,-0.28126368,-0.24774148,0.33675805,0.03746164,0.12733914,0.09214106,-0.35317972,0.49345806,-0.14484765,0.06152966,-0.18431777,-0.048113223,-0.47544786,-0.33898163,0.32528985,-0.22873995,-0.17303172,0.21048632,-0.08506107,-0.24107367,-0.1332848,0.35740942,-0.5896629,-0.41277078,-0.38885683,0.24234755,0.13208888,0.043683235,0.1780711,-0.014218416,0.11314606,-0.09209795,-0.06444193,0.6316962,0.16790925,-0.023901109,-0.10224754,0.0060919034,0.19646695,-0.2588165,0.18812367,0.02685671,0.33576748,0.018414523,-0.10306445,0.040253866,-0.22638129,9.505736e-06,0.182601,-0.039122835,-0.24748954,0.017074706,-0.4463301,-0.16213425,0.050018035,0.176752,0.024563346,0.0026132888,0.10159111,0.001136446,-0.025208581,-0.12682623,0.22581628,0.12291578,0.17176865,0.45798582,-0.0074094348,-0.1800068,0.2891869,0.2993622,-0.14954627,-0.38952777,-0.30715564,-0.1336829,0.06888456,0.48386496,-0.53245175,-0.093996726,0.4310551,0.028261948,0.06462729,0.052500915,0.322774,0.0040936824,-0.028114218,0.0043976437,-0.40388465,-0.62312067,0.0054867873,0.021131072,-0.22566265,0.28742617,0.028569944,-0.14090697,0.025195725,-0.4802191,0.11339196,-0.16535297,0.008107334,0.1090163,-0.20554504,0.11734341,0.4095185,0.36999732,0.046569794,-0.44063658,-0.041318234,-0.23075488,0.26295823,-0.090015635,0.15232031,-0.02559941,-0.019115433,-0.02622822,0.489721,-0.26801893,0.0094079655,0.5877372,0.077201515,-0.12527047,-0.16018279,-0.08659932,-0.113133416,-0.066702336,-0.3833209,0.16755989,-0.34686008,0.12585077,-0.3714205,-0.039465822,0.35411605,-0.078134954,0.1502911,0.29366747,0.11577656,-0.13633683,-0.024788715,-0.039773256,-0.28604612,0.25801733,0.09769797,-0.030820915,-0.16255368,0.34112757,0.07880175,0.021094581,-0.409878,0.095827945,0.04422169,0.001037882,-0.19427851,0.21324477,0.3553692,-0.37727073,0.0016180893,0.10438395,0.15830016,0.035718348,0.06841929,0.32014412,0.21020226,-0.26254645,0.10995928,0.013432503,0.005325271,-0.25236946,0.22287105,0.09707114,0.16654517,0.11032792,0.09697536,0.21541682,-0.28829533,-0.009343898,0.19034882,-0.024213359,0.29945758,-0.019570459,0.040527496,0.50526553,-0.16046746,-0.14347154,0.017736424,-0.038467854,-0.1333831,0.40344927,-0.20134188,0.06751333,-0.0064291395,0.15153243,-0.006441163,-0.15362448,0.36816782,-0.59554803,-0.18605942,0.5422837,-0.05141622,0.05064855,0.34921545,-0.08176005,-0.5551801,-0.08147682,-0.08744775,0.47462758,0.043138772,0.0017587969,0.12614264,0.39291647,-0.024234092,-0.29022276,0.23137024,-0.04660919,0.058404036,0.137952,0.1044107,0.22539766,-0.33025146,0.04624822,-0.015728254,-0.078336276,-0.12175391,0.2454174,-0.35853982,-0.5106406,-0.27245012,-0.28451347,-0.23424643,-0.02729399,-0.006065087,-0.06108637,0.06575562,0.12881398,0.10389899,-0.30602875,0.20364666,0.21165738,-0.0041915476,0.13722631,-0.3164104,0.10149926,0.08317907,0.08228338,-0.17569003,-0.06923973,0.010013602,0.3395095,0.13188943,0.15193653,0.15987054,-0.0017275419,0.19972673,0.026965357,-0.023522718,-0.006764924,-0.24158125,0.1915091,-0.20699558,-0.29009256,-0.21520948,0.024718069,-0.18848029,0.15039104,-0.0011711488,0.10290854,0.12639704,-0.039789323,0.11118279,-0.34227067,0.009994839,-0.022541797,0.004464198,-0.24575661,0.08877347,0.07865475,0.1826205,0.0053493213,0.011450974,-0.4279052,-0.110387005,0.025462572,0.04026869,0.26243946,0.002812747,0.055570755,-0.04447361,-0.09408983,-0.25625747,-0.30541816,0.043545976,-0.26014918,0.140537,-0.48640025,-0.060132407,0.061768096,0.17183188,0.06704039,0.083998054,0.26716056,-0.31624907,0.25614715,0.11682943,0.1265741,-0.03892296,-0.040705655,-0.11338223,0.06697352,-0.03942586,0.17063843,0.15511379,-0.47434312,0.3091403,0.114928335,-0.13300864,0.07413602,0.22636676,0.11999537,-0.37126872,0.11607906,-0.08262735,0.0824198,0.11139387,0.07526435,-0.29171023,0.041055534,-0.61836845,0.09817044,-0.06525552,0.008942078,0.11651012,-0.15387972,0.33332837,0.043309614,0.24526107,-0.09742984,8.169003e-05,0.0034550838,0.33575007,0.3172099,-0.27296725,0.06998822,0.019964378,-0.33780187,-0.09563615,-0.13221629,-0.02532386,-0.36532143,0.27787453,-0.31755218,-0.012262182,0.23725949,0.3305575,0.30918092,0.16437516,-0.047777746,0.024981964,0.3381303,-0.1961493,0.0016496857,0.09544582,0.14247446,-0.12450591,-0.03627339,-0.060326062,-0.067283496,-0.019981999,0.1446967,0.02882808,-0.112932295,-0.07226729,-0.08054878,0.08050939,-0.28853253,-0.049018044,0.017855128,-0.14713997,0.2130142,-0.27344495,0.013433746,0.13182205,-0.2691713,0.1688016,0.075026944,-0.024904102,0.10077347,0.028687298,-0.090598255,-0.2618535,-0.21260035,-0.20195165,0.36319998,0.02674682,-0.1020499,-0.123511724,-0.003642102,-0.3997665,-0.0143390875,-0.123899035,-0.19839063,-0.039451882,0.09349331,-0.022790188,0.35922238,0.29378068,0.057812087,-0.018851597,0.01917857,-0.32583597,0.0715897,0.0027700365,-0.28314838,0.12091403,-0.09047323,-0.062074956,-0.103149466,0.3000564,0.10949193,-0.3436343,-0.15816714,0.14746052,-0.52673835,0.0089369975,0.1561368,0.25593814,-0.25861138,-0.014260302,0.18559745,-0.14571889,0.20796634,0.17954817,-0.09771016,-0.17375958,0.4153521,-0.1592597,0.07269709,0.01280562,0.092697136,-0.08966824,0.1364662,0.29190716,0.2725758,-0.21688354,-0.051513985,-0.23736648,-0.06738123,-0.14262995,-0.17216651,-0.06889763,0.10485046,0.43257862,0.19682729,0.20533523,-0.09715749,0.1359377,0.093902595,0.024687126,-0.27759254,0.012827594,0.015733853,-0.1271841,-0.30871192,-0.22971153,0.34938127,0.14387979,-0.30876085,-0.27403048,-0.38553524,0.01901656,0.23803659,0.08601224,-0.25166866,0.50184774,0.11659524,-0.10041675,0.023878094,0.1261304,0.2469206,-0.1603253,0.20139372,-0.14338861,-0.21664967,-0.5400088,-0.29845905,-0.2801851,-0.09056196,-0.20545754,0.1191909,-0.09414974,0.23914169,0.088611335,-0.29993615,-0.20697528,0.32960787,-0.0022915006,-0.010624807,-0.0010209326,-0.5450774,-0.11091987,0.006158771,-0.32387123,0.4269025,0.07165812,0.18984169,-0.18546435,-0.07867424,0.11551718,0.21479647,-0.052868318,-0.5362675,-0.23527318,-0.064288914,0.09851214,-0.30383822,-0.05563345,0.18781659,-0.08728603,-0.22874618,0.038494598,-0.121195756,-0.011552804,0.10425499,-0.059540868,-0.55104,-0.16754992,0.15843326,0.26647344,-0.07160836,-0.102565184,-0.078116916,-0.13241357,0.067446254,-0.23650974,0.021293778,0.028032528,-0.06191862,0.19875762,-0.3308428,0.27365232,-0.22306263,-0.052122727,-0.13700342,-0.11603968,0.49895167,-0.05577561,-0.30314657,-0.23989922,-0.13112259,-0.23447202,0.12855355,-0.046125263,-0.15017186,-0.10273955,-0.11516408,0.32326,0.13429663,-0.0476944,-0.030667584,0.3711514,-0.24309646,-0.12002885,-0.035996914,0.05729208,0.21520495,-0.028011803,0.08549711,-0.12984632,0.41246185,-0.075884916,0.07743125,0.38654706,0.022571862,-0.12478354,-0.038945895,-0.056611594,-0.26671976,-0.037662137,0.0041146223,-0.25110552,-0.060501467,-0.10828337,0.16380706,0.120212145,-0.09759073,0.07492847,-0.16729905,0.09873296,0.22865658,-0.21015252,-0.003535322,-0.16328748,0.033011388,-0.031755906,0.11541102,-0.17477027,-0.10023304,-0.15813512,0.28706706,-0.1312869,-0.20866922,0.060617782,0.19340003,-0.033450685,-0.15856099,-0.21045357,-0.109759346,-0.2623619,0.33853915,-0.065591,0.062091332,0.14468798,-0.18964645,-0.14908439,0.053408016,0.15031144,0.15000208,-0.019388601,0.09350036,-0.09868962,-0.107415766,0.059099656,-0.28382304,-0.35811105,-0.45414507,0.061754826,0.3289415,-0.053728763,-0.2255229,-0.053469595,0.25126642,0.31456673,-0.063166805,-0.14634669,-0.06101463,-0.15890995,0.01359682,-0.0056105684,0.097725295,0.04287555,-0.4234043,-0.10727902,0.13561115,-0.16679966,0.067033656])

1536

Let's setup the new chain using Aurora as the vector store:

In [30]:
chain = (
    {"context": db.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

response = chain.invoke("What are the main challenges and advantages of doing business in India, including insights on market sensitivity, price, and speed of decision-making?")

In [31]:
print(response)

Based on the context provided, some of the main challenges and advantages of doing business in India mentioned are:

Challenges:
1. The Indian market is very price sensitive. Indian customers tend to focus a lot on cost and go for the cheapest option, sometimes at the expense of quality or efficiency.
2. Many Indian companies try to do everything themselves instead of buying services, even when external help could be beneficial. This is driven by the desire to save costs.
3. There is too much focus on saving money rather than investing in things that could improve sales and operations like good websites, UI/UX, efficient processes etc.
4. Marketing to Indian businesses is a challenge since the company was previously known more among developers than businessmen.

Advantages:
1. India is a fast deciding market compared to other regions like Europe where decision cycles can be very long.
2. Indian companies want to move with speed.
3. The large population provides access to affordable lab

## Resources 

- [Building a RAG application from scratch using Python - By Santiago](https://www.youtube.com/watch?v=BrsocJb-fAo&t=548s&ab_channel=Underfitted)
- [Vector Embeddings and RAG Demystified: Leveraging Amazon Bedrock, Aurora, and LangChain - Part 1](https://community.aws/content/2gvh6fQM4mJQduLye3mHlCNvPxX/vector-embeddings-and-rag-demystified)
- [Vector Embeddings and RAG Demystified: Leveraging Amazon Bedrock, Aurora, and LangChain - Part 2](https://community.aws/content/2gvh8oJzNrM4vxdZDd903zcEFJc/vector-embeddings-and-rag-demystified-2)